https://github.com/kushalj001/pytorch-question-answering/blob/master/1.%20DrQA.ipynb

In [12]:
import pandas as pd
import numpy as np
import torchtext
import torch
from torch import nn
import json, re, unicodedata, string, typing, time
import torch.nn.functional as F
import spacy
from collections import Counter
import pickle
import Bert_Question_Answering_preprocessing as BQ
from nltk import word_tokenize
nlp = spacy.load('en')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
train_data = BQ.load_json(r'‪C:\Users\admin\jupyter\pytorch\Bert\squad_train.json')
valid_data = BQ.load_json(r'‪C:\Users\admin\jupyter\pytorch\Bert\squad_dev.json')

train_list = parse_data(train_data)
valid_list = parse_data(valid_data)

print('Train list len : ', len(train_list))
print('Valid list lne : ', len(valid_list))

train_df = pd.DataFrame(train_list)
valid_df = pd.DataFrame(valid_list)

OSError: [Errno 22] Invalid argument: '\u202aC:\\Users\\admin\\jupyter\\pytorch\\Bert\\squad_train.json'